In [ ]:

import math
from typing import List

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample_poly

from inference import dataset, experiments, raw, fir_filter, iir_filter

%matplotlib qt


## Load raw ECG

In [ ]:

signal_path: str = './Delineation_Datasets/AttCHR0102rcs4_Sinius/13-20-54.EDF'
ecg, old_fs = raw.load_data(signal_path)


## Load processed ECG

In [ ]:

ecg_data = pd.read_csv('./Delineation_Datasets/AttCHR0102rcs4_Sinius/13-20-54_data.csv')
ecg_signal = ecg_data['signal'].to_numpy() * 1_000


## Resample signals

In [ ]:

new_fs = 360
res_ecg_raw = resample_poly(ecg, new_fs, old_fs)
res_ecg_signal = resample_poly(ecg_signal, new_fs, old_fs)


## Use trained model

In [ ]:

num_examples = res_ecg_raw.shape[0] // 512
res_ecg_raw_input = res_ecg_raw[:num_examples*512].reshape(-1, 512, 1)


In [ ]:

noise_version = 1
experiment = experiments.Experiment.Multibranch_LANLD
model = experiments.load_model(experiment, noise_version)


In [ ]:

res_ecg_raw_preds = model.predict(res_ecg_raw_input, batch_size=32, verbose=1)


In [ ]:

start = 0
end = 100_000

temp_preds = res_ecg_raw_preds.reshape(-1)

plt.figure()
plt.title(f'{experiment.value}')
plt.axhline(y=0)
plt.plot(res_ecg_raw[start:end], label='raw')
plt.plot(temp_preds[start:end], label='pred')
plt.plot(res_ecg_signal[start:end], label='signal')
plt.legend()
plt.show()


## Use FIR

In [ ]:

Fc_l = 0.67
Fc_h = 100.0
Fs = new_fs
res_ecg_fir, N = fir_filter.FIRRemoveBL(res_ecg_raw, Fs, Fc_l, 4.5)
res_ecg_fir, N = fir_filter.FIRRemoveHF(res_ecg_fir, Fs, Fc_h, 4.5)


In [ ]:

plt.figure()
plt.title('FIR')
plt.plot(res_ecg_raw[start:end], label='raw')
plt.plot(res_ecg_fir[start:end], label='fir')
plt.plot(res_ecg_signal[start:end], label='signal')
plt.axhline(y=0, lw=0.6, ls="--")
plt.legend()
plt.show()


## Use IIR

In [ ]:

Fc_l = 0.67
Fc_h = 100.0
Fs = new_fs
res_ecg_iir = iir_filter.IIRRemoveBL(res_ecg_raw, Fs, Fc_l)
res_ecg_iir = iir_filter.IIRRemoveHF(res_ecg_iir, Fs, Fc_h)


In [ ]:

plt.figure()
plt.title('IIR')
plt.plot(res_ecg_raw[start:end], label='raw')
plt.plot(res_ecg_iir[start:end], label='iir')
plt.plot(res_ecg_signal[start:end], label='signal')
plt.axhline(y=0, lw=0.6, ls="--")
plt.legend()
plt.show()
